# Heuristics vs Mathematical Optimization Using the Binary Paintshop Problem
- In car manufacturing one of the final production steps is painting.
- Multiple cars of different types (A to D) arrive in a given sequence at the paintshop.
 ![sequence](car_sequence.png)
- The cars have to be painted with a base coat that is either white or black (here referred to as red or blue).
- The demand for white and black colors for a given car type is also given.

This problem can be simplified to a minimal working example:
- In the sequence of cars arriving at the paintshop each vehicle type arrives exactly twice.
- One car of each vehicle type has to be painted white, the other one has to be painted black.

As changing colors requires time and produces waist, the goal is to minimize the number of color changes with respect to the constraint of coloring one car white and one black for each vehicle type.

This problem can be solved both heuristically or with a mathematical optimization approach.

In [1]:
# Install dependencies
! pip install -q gamspy

In [2]:
import random
random.seed(16)

## Data

In [3]:
# n_types = 6
# types = list(string.ascii_uppercase)[:n_types]
# sequence = random.sample(types, counts=[2 for n in range(n_types)], k=n_types * 2)
# sequence

In [4]:
sequence = ["A", "D", "E", "B", "A", "F", "C", "B", "C", "D", "E", "F"]
types = set(sequence)
n_types = len(types)
n_types

6

## Heuristic
- Start to color every vehicle type with color red
- Continue to use red as long as possible
- Than switch color until every car is painted

In [5]:
changes = 0
colors = {"white": set(), "black": set()}
result = []


def paint_car(colors_dict, result_list, color, car_type):
    colors_dict[color].add(car_type)
    result_list.append(color)


current_color = "white"
for car in sequence:
    if car not in colors[current_color]:
        paint_car(colors, result, current_color, car)
    else:
        # change color
        changes += 1
        if current_color == "white":
            current_color = "black"
        else:
            current_color = "white"
        paint_car(colors, result, current_color, car)

print(result)
print("Number of changes:", changes)

['white', 'white', 'white', 'white', 'black', 'black', 'black', 'black', 'white', 'black', 'black', 'white']
Number of changes: 4


## Mathematical Modeling

In [6]:
import gamspy as gp
from gamspy.math import sqr

# create container
m = gp.Container()

# create sets
i = gp.Set(m, "i", description="number in sequence")
j = gp.Set(m, "j", description="car type")
IJ = gp.Set(
    m,
    "IJ",
    domain=[i, j],
    records=[(i + 1, sequence[i]) for i in range(len(sequence))],
    domain_forwarding=True,
)

# create variables
X = m.addVariable("X", domain=[i], type="binary", description="color indicator")

--- Warning: The GAMS version (46.4.0) differs from the API version (46.3.0).
--- Warning: The GAMS version (46.4.0) differs from the API version (46.3.0).
--- Warning: The GAMS version (46.4.0) differs from the API version (46.3.0).


In [7]:
obj = gp.Sum(i, sqr(X[i] - X[i.lead(1)]))

equation_two = gp.Equation(container=m, name="equation_two", domain=j, description="Ensure that each position i is painted white exactly once.")
equation_two[j] = gp.Sum(IJ[i, j], X[i]) == 1

paintshop = gp.Model(
    container=m,
    name="paintshop",
    equations=[equation_two],
    problem="MINLP",
    sense=gp.Sense.MIN,
    objective=obj,
)

paintshop.solve(solver="sbb")

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,Integer,2,7,13,MINLP,SBB,0.266


In [8]:
opt_changes = paintshop.objective_value
opt_changes

2.0

In [9]:
X.records["level"]

0     0.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
Name: level, dtype: float64

## Compare

### Optimization

In [10]:
# '\033[91m' - red / white
# '\033[94m' - blue / black

# define red color
if round(X.records["level"][0], 1) == 0:
    colors = {0: "\033[91m", 1: "\033[94m"}
else:
    colors = {1: "\033[91m", 0: "\033[94m"}

opt_coloring = []
for x in range(len(sequence)):
    opt_coloring.append(colors[round(X.records["level"][x], 1)])
    opt_coloring.append(sequence[x])

print(f"Using the optimization approach {round(opt_changes)} color changes are needed.")
print("The coloring sequence is:", "".join(opt_coloring))

Using the optimization approach 2 color changes are needed.
The coloring sequence is: ADEBAFCBCDEF


### Heuristic

In [11]:
coloring = []
for x in range(len(sequence)):
    if result[x] == "white":
        c = "\033[91m"
    else:
        c = "\033[94m"
    coloring.append(c)
    coloring.append(sequence[x])
print(f"Using the heuristic approach {changes} color changes are needed.")
print("The coloring sequence is:", "".join(coloring))

Using the heuristic approach 4 color changes are needed.
The coloring sequence is: ADEBAFCBCDEF


## Transfer to the Real World - Multi Vehicle Paintshop Problem
The presented problem is a very easy and simplified version of the real problem where a given number of vehicles of different type arrive at a given sequence at the paint shop and a given share of each vehicle type has to be painted black and the rest white. However, the simplified version gives us a slight impression of how powerful mathematical optimization is. 
Solving the real (multi vehicle paint shop problem) is a more complicated version of the presented (binary paint shop problem).  

## Data

In [12]:
sequence = random.choices(list(types), k=128)
demand_white = {t: random.randint(0, sequence.count(t)) for t in types}
demand_black = {t: sequence.count(t) - demand_white[t] for t in types}
demand_white

{'E': 16, 'F': 4, 'B': 15, 'C': 2, 'D': 3, 'A': 10}

## Heuristic

In [13]:
changes = 0
colors = {"white": dict(demand_white), "black": dict(demand_black)}
result = []


def paint_car(colors_dict, result_list, color, car_type):
    colors_dict[color][car_type] -= 1
    result_list.append(color)


current_color = "white"
for car in sequence:
    if colors[current_color][car] > 0:
        paint_car(colors, result, current_color, car)
    else:
        # change color
        changes += 1
        if current_color == "white":
            current_color = "black"
        else:
            current_color = "white"
        paint_car(colors, result, current_color, car)

print(result)
print("Number of changes:", changes)

['white', 'white', 'white', 'white', 'white', 'white', 'white', 'white', 'white', 'white', 'white', 'white', 'white', 'white', 'white', 'white', 'white', 'white', 'white', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'black', 'white', 'black', 'white', 'black', 'white', 'black', 'black', 'black', 'white', 'black', 'white', 'white', 'white', 'white', 'white', 'white', 'black', 'white', 'white', 'white', 'black', 'black', 'white', 'black', 'white', 'black', 'white', 'white', 'white', 'white', 'white', 'black', 'white', 'white', 'white', 'black', 

## Model

In [14]:
IJ.setRecords([(i + 1, sequence[i]) for i in range(len(sequence))])

# create parameters
white_demand = gp.Parameter(
    m,
    "white_demand",
    domain=[j],
    records=[(type, demand) for type, demand in demand_white.items()],
)
black_demand = gp.Parameter(
    m,
    "black_demand",
    domain=[j],
    records=[(type, demand) for type, demand in demand_black.items()],
)

Z = gp.Variable(m, "Z", type='binary', domain=i, description='Indicates a color change from i to i+1')

Note that the following notation is a linearization of $(X_i - X_{i+1})^2$

In [15]:
LinearizeObjective1 = gp.Equation(m, "LinearizeObjective1", domain=i)
LinearizeObjective1[i] = Z[i] >= X[i] - X[i.lead(1)]

LinearizeObjective2 = gp.Equation(m, "LinearizeObjective2", domain=i)
LinearizeObjective2[i] = Z[i] >= X[i.lead(1)] - X[i]

MeetBlackDemand = gp.Equation(m, "MeetBlackDemand", domain=j)
MeetBlackDemand[j] = gp.Sum(IJ[i, j], X[i]) == black_demand[j]

MeetWhiteDemand = gp.Equation(m, "MeetWhiteDemand", domain=j)
MeetWhiteDemand[j] = gp.Sum(IJ[i, j], 1 - X[i]) == white_demand[j]

multi_paintshop = gp.Model(
    m,
    "MultiPaintshop",
    equations=[LinearizeObjective1, LinearizeObjective2, MeetWhiteDemand, MeetBlackDemand],
    problem="MIP",
    sense=gp.Sense.MIN,
    objective=gp.Sum(i, Z[i]),
)

multi_paintshop.solve(solver="cplex")

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,11,269,257,MIP,CPLEX,0.093


In [16]:
opt_changes = multi_paintshop.objective_value
opt_changes

11.0

## Compare

### Mathematical Model

In [17]:
# '\033[91m' - red
# '\033[94m' - blue
colors = {1: "\033[91m", 0: "\033[94m"}

opt_coloring = []
for x in range(len(sequence)):
    opt_coloring.append(colors[round(X.records["level"][x], 1)])
    opt_coloring.append(sequence[x])

print(f"Using the optimization approach {round(opt_changes)} color changes are needed.")
print("The coloring sequence is:", "".join(opt_coloring))

Using the optimization approach 11 color changes are needed.
The coloring sequence is: BBBBBCFCEFBEDFDAFADDEBCFFFFDBBCAFEADDCFEFBEDDEABABDBCBEEFBBCCBFCADCDFEEEECAEDADECFFECAEEBEBDBEEFDACEFBEEBAFAABFFDDAFBEFFEEAFBCBC


### Heuristic

In [18]:
coloring = []
for x in range(len(sequence)):
    if result[x] == "white":
        c = "\033[91m"
    else:
        c = "\033[94m"
    coloring.append(c)

    coloring.append(sequence[x])
print(f"Using the heuristic approach {changes} color changes are needed.")
print("The coloring sequence is:", "".join(coloring))

Using the heuristic approach 31 color changes are needed.
The coloring sequence is: BBBBBCFCEFBEDFDAFADDEBCFFFFDBBCAFEADDCFEFBEDDEABABDBCBEEFBBCCBFCADCDFEEEECAEDADECFFECAEEBEBDBEEFDACEFBEEBAFAABFFDDAFBEFFEEAFBCBC
